In [3]:
import pandas as pd
from datetime import date, time, datetime, timedelta
from tqdm.auto import tqdm

# считваем только нужные колонки и  колонку даты парсим как дату.
data = pd.read_csv(r'path\data.csv', sep=';', usecols=['userid','timestamp'], 
                   parse_dates=['timestamp'])
data = data.sort_values('timestamp')

# функция возвращает ДФ содержащий по конкретному юзеру номера сессиий с их началом и концом.
def get_user_sessions(df):
#   просто делаем копию df1 = df.copy(), чтобы пандас не ругался на попытки изменить основной датафрейм
    df1 = df.copy()
    df1['prev_timestamp'] = df1['timestamp'].shift(1)
    
    min_time = df1['timestamp'].min()
    max_time = df1['timestamp'].max()
    
    df1['time_since_prev'] = df1['timestamp'] - df1['prev_timestamp']
    
    df1['new_session'] = df1.apply(lambda x: 1 if (x['time_since_prev'] > timedelta(minutes=30) 
                                                 or x['timestamp']==min_time) else 0, axis=1)
    
    df1['session_num'] = df1['new_session'].cumsum()
    
    df_session_start = df1.groupby(['session_num'], as_index=False) \
            .agg({'timestamp': 'min'}).rename(columns={'timestamp': 'start'})
    df_session_end = df1.groupby('session_num', as_index=False) \
            .agg({'timestamp': 'max'}).rename(columns={'timestamp': 'end'})
    
    df_session_end['end'] = df_session_end['end'] + timedelta(minutes=30)
    
    df_sessions = df_session_start.merge(df_session_end, on='session_num')
    
    return df_sessions

# функция возвращает ДФ в котором чередуются строки начала и конца сессий с указанием времени
def get_sessions_number(df):
    df_prep = df[['start', 'end']].melt()

    df_prep['flow'] = df_prep.apply(lambda x: 1 if x['variable']=='start' else -1, axis=1)

    df_prep = df_prep.sort_values('value')
    
    df_prep['sessions_number'] = df_prep['flow'].cumsum()
    
    df_prep['sessions_number_prev'] = df_prep['sessions_number'].shift(1)

    df_res = df_prep[(df_prep['sessions_number'] == df_prep['sessions_number'].max()) |
                        (df_prep['sessions_number_prev'] == df_prep['sessions_number'].max())]
    
    return df_res[['variable', 'value']]


users_list = list(data['userid'].unique())

df_sessions_res = pd.DataFrame()

for i in range(len(users_list)):
    df_sessions_temp = get_user_sessions(data[data['userid']==users_list[i]])
    df_sessions_res = pd.concat([df_sessions_res, df_sessions_temp])

df2 = get_sessions_number(df_sessions_res).reset_index(drop=True)
for i in tqdm(range(0, len(df2['value'])-1, 2)):
    start = df2['value'][i]
    end = df2['value'][i+1]
    print(f"{start} {end}")

2022-08-22 10:04:00 2022-08-22 11:15:00
2022-08-22 12:14:00 2022-08-22 12:44:00

Wall time: 73 ms
